<a href="https://colab.research.google.com/github/annakalinina18/star-fle/blob/main/annotation_avec_LLM/mistral_large_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from tqdm import tqdm
from openai import OpenAI

# =========================
# 0. CLIENT
# =========================

client = OpenAI(
    api_key="",
    base_url="https://api.mistral.ai/v1"
)

# =========================
# 1. MODÈLE
# =========================

MODEL_NAME = "mistral-large"

# =========================
# 2. BASELINE PROMPT (segmentation)
# =========================

BASELINE_PROMPT = """
Tu es linguiste.

Ta tâche est de décider si l’expression fournie
constitue une expression polylexicale figée
ou une combinaison libre.

Définitions :

Expression_polylexicale :
Suite de mots formant une unité relativement figée,
conventionnelle ou lexicalisée.

Combinaison_libre :
Suite de mots construite librement en discours,
sans figement particulier.

Réponds UNIQUEMENT selon le format :

Décision : <Expression_polylexicale | Combinaison_libre>
Explication : <une phrase courte>

Expression : {expression}
Contexte : {contexte}
"""

ALLOWED = {
    "Expression_polylexicale",
    "Combinaison_libre",
}

# =========================
# 3. APPEL LLM
# =========================

def classify_baseline(expression, examples):
    contexte = "" if examples is None or pd.isna(examples) else str(examples)

    prompt = BASELINE_PROMPT.format(
        expression=str(expression).strip(),
        contexte=contexte
    )

    resp = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
        max_tokens=80,
    )

    text = (resp.choices[0].message.content or "").strip()

    decision = None
    for line in text.splitlines():
        if line.lower().startswith("décision"):
            decision = line.split(":", 1)[-1].strip()
            break

    if decision not in ALLOWED:
        text = (
            "Décision : INVALID\n"
            "Explication : Réponse hors format.\n\n"
            + text
        )

    return text

# =========================
# 4. TRAITEMENT EXCEL
# =========================

input_file = "segmentation_input.xlsx"
df = pd.read_excel(input_file)
df["llm_raw_response"] = None

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Baseline segmentation"):
    expr = row.get("expression")
    ex = row.get("examples_joined")

    if pd.isna(expr) or not str(expr).strip():
        df.at[idx, "llm_raw_response"] = "N/A"
        continue

    df.at[idx, "llm_raw_response"] = classify_baseline(expr, ex)

output_file = "annotated_segmentation_baseline_mistral.xlsx"
df.to_excel(output_file, index=False)

print(f"Saved: {output_file}")
